In [1]:
import os 
import sys 

sys.path.insert(0, '..')

In [2]:
from bio_check import Verifier
from pprint import pp

In [3]:
sbml_fp = "../model-examples/sbml-core/Elowitz-Nature-2000-Repressilator/BIOMD0000000012_url.xml"
omex_fp = sbml_fp.replace("/BIOMD0000000012_url.xml", ".omex")
report_fp = omex_fp.replace(".omex", "/reports.h5")
duration = 10
nsteps = 100 
comparison_id = "test_notebook_sbml0"
simulators = ["copasi", "tellurium"]
selection_list = ["cI mRNA"]

In [4]:
verify = Verifier()

{'bio-check-message': 'Hello from the Verification Service API!'}


In [5]:
# inspect public methods and attributes

_dir = dir(verify)

for m in _dir:
    print(m) if not m.startswith("_") else None

data
endpoint_root
export_csv
get_compatible
get_verify_output
select_observables
verify_omex
verify_sbml
visualize


In [6]:
# submit a new verification 

submission_1 = verify.verify_sbml(sbml_filepath=sbml_fp, comparison_id=comparison_id, duration=duration, number_of_steps=nsteps, simulators=simulators)

In [7]:
submission_1

RequestError(error='422 Client Error: Unprocessable Entity for url: https://biochecknet.biosimulations.org/verify-sbml')

In [8]:
submission_2 = verify.verify_omex(omex_filepath=omex_fp, comparison_id=comparison_id, truth=report_fp)

Selection list: None


In [9]:
submission_2

{'job_id': 'utc_comparison_01936049-81b2-4ee2-9b05-472cd45a8e46',
 'status': 'PENDING',
 'timestamp': '2024-07-31 17:42:18.467037',
 'comparison_id': 'utc_comparison',
 'path': 'uploads/utc_comparison_01936049-81b2-4ee2-9b05-472cd45a8e46/Elowitz-Nature-2000-Repressilator.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'ground_truth_report_path': None,
 'include_output': True,
 'rTol': None,
 'aTol': None,
 'selection_list': None}

In [10]:
# use job id generated from submission to poll gateway

omex_job_id = submission_2['job_id']

verify.get_verify_output(omex_job_id)

{'content': {'job_id': 'utc_comparison_01936049-81b2-4ee2-9b05-472cd45a8e46',
  'status': 'IN_PROGRESS',
  'timestamp': '2024-07-31 17:42:19.492427',
  'comparison_id': 'utc_comparison',
  'worker_id': 'ac8a23ef-d810-4a07-9222-2e096adf0ad4'}}

In [11]:
# we can call this multiple times until ready

verify.get_verify_output(omex_job_id)

{'content': {'job_id': 'utc_comparison_01936049-81b2-4ee2-9b05-472cd45a8e46',
  'status': 'IN_PROGRESS',
  'timestamp': '2024-07-31 17:42:19.492427',
  'comparison_id': 'utc_comparison',
  'worker_id': 'ac8a23ef-d810-4a07-9222-2e096adf0ad4'}}

In [12]:
output = verify.get_verify_output(omex_job_id)

In [13]:
pp(output)

{'content': {'job_id': 'utc_comparison_01936049-81b2-4ee2-9b05-472cd45a8e46',
             'status': 'IN_PROGRESS',
             'timestamp': '2024-07-31 17:42:19.492427',
             'comparison_id': 'utc_comparison',
             'worker_id': 'ac8a23ef-d810-4a07-9222-2e096adf0ad4'}}


In [14]:
# use selection list to return only one observable

selection_list = ["cI mRNA"]

submission_3 = verify.verify_omex(omex_filepath=omex_fp, comparison_id="notebook_test_omex2", truth=report_fp, selection_list=selection_list)

Selection list: ['cI mRNA']


In [15]:
submission_3

{'job_id': 'utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9',
 'status': 'PENDING',
 'timestamp': '2024-07-31 17:42:39.985493',
 'comparison_id': 'utc_comparison',
 'path': 'uploads/utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9/Elowitz-Nature-2000-Repressilator.omex',
 'simulators': ['amici', 'copasi', 'tellurium'],
 'ground_truth_report_path': None,
 'include_output': True,
 'rTol': None,
 'aTol': None,
 'selection_list': None}

In [16]:
output2 = verify.get_verify_output(submission_3['job_id'])

In [17]:
output2

{'content': {'status': 'PENDING',
  'job_id': 'utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9',
  'path': 'uploads/utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'comparison_id': 'utc_comparison',
  'timestamp': '2024-07-31 17:42:39.985493',
  'ground_truth_report_path': None,
  'include_outputs': True,
  'rTol': None,
  'aTol': None,
  'selection_list': None}}

In [19]:
output = None 
while isinstance(output, type(None)):
    output2 = verify.get_verify_output(submission_3['job_id'])
    if output2['content']['status'] == "COMPLETED":
        output = output2

KeyboardInterrupt: 

In [20]:
verify.get_verify_output(submission_3['job_id'])

{'content': {'status': 'PENDING',
  'job_id': 'utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9',
  'path': 'uploads/utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'comparison_id': 'utc_comparison',
  'timestamp': '2024-07-31 17:42:39.985493',
  'ground_truth_report_path': None,
  'include_outputs': True,
  'rTol': None,
  'aTol': None,
  'selection_list': None}}

In [21]:
verify.get_verify_output(submission_3['job_id'])

{'content': {'status': 'PENDING',
  'job_id': 'utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9',
  'path': 'uploads/utc_comparison_d8389eed-bce2-47d8-894c-9d7519dbc2b9/Elowitz-Nature-2000-Repressilator.omex',
  'simulators': ['amici', 'copasi', 'tellurium'],
  'comparison_id': 'utc_comparison',
  'timestamp': '2024-07-31 17:42:39.985493',
  'ground_truth_report_path': None,
  'include_outputs': True,
  'rTol': None,
  'aTol': None,
  'selection_list': None}}